# **Deep Neural Network for MNIST**

Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs).

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image).

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes.

Our goal would be to build a neural network with 2 hidden layers.

## **Import the relevant packages**

In [10]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## **Data**

In [11]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [17]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)

def scale(image, label):
  image = tf.cast(image, tf.float32) # 转换为 float32，float是带小数点，精度大约是小数点后7位有效数字。
  image /= 255.                      # 像素值缩放到 [0,1]
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(BATCH_SIZE)

validation_inputs, validation_targets = next(iter(validation_data))

**tfds.load()** tfds = TensorFlow Datasets 库，里面收录了很多常见的数据集（MNIST, CIFAR-10, IMDB, Fashion-MNIST 等）。

tfds.load() 会自动下载并准备数据集。

**name='mnist'**
指定要下载的数据集是 MNIST。

**with_info=True**
返回一个 tfds.core.DatasetInfo 对象（这里是 mnist_info），里面包含：

数据集的元信息（训练集大小、测试集大小、类别标签数、图像大小等）。

例如 mnist_info.splits['train'].num_examples 可以得到训练样本数 = 60000。

**as_supervised=True**
数据集会以 (input, label) 形式返回。

input：28×28 灰度图（Tensor）。

label：对应的数字（0–9 的整数）。
如果不用 as_supervised，返回的是字典格式（更冗余）。

**mnist_train** → 训练集 (60,000 张)。

**mnist_test** → 测试集 (10,000 张)。

MNIST 训练集共有 60,000 张。

**num_validation_samples** 取 10% (6,000 张) 作为验证集。

**tf.cast(..., tf.int64):** 确保数据类型是 TensorFlow 的 int64（方便后面 take() 和 skip() 用）。

**int64:** 64 位整数。把 6000.0 转换为 TensorFlow 的 64 位整数，变成 6000。

**mnist_train.map(scale), mnist_test.map** 用map()将mnist_train里面的数据通过scale function转换成float32，缩放到[0,1]

**shuffle(BUFFER_SIZE)：** 把数据打乱，避免模型训练时顺序带来偏差。

**BUFFER_SIZE=10000：** 并不是把所有数据一次性打乱，而是维持一个 缓冲区，里面随机采样数据。一般 BUFFER_SIZE 越大，打乱效果越好（但更耗内存）。

**take(num_validation_samples)：** 取前 num_validation_samples（6000） 个样本作为验证集。

**skip(num_validation_samples)：** 跳过这部分，剩下的作为训练集（54000）。

因为之前做过 shuffle，所以 取前6000并不会有顺序偏差，相当于随机划分。

**train_data.batch(100)：训练集**会被分成 每批100个样本，比如 (100, 28, 28, 1) → (100 个图片, 每个28×28, 单通道)。

**validation_data.batch(num_validation_samples)：验证集**直接设为 一个大批次（6000个），这样在验证时就能一次性取出所有验证数据。

**test_data.batch(BATCH_SIZE)测试集**也是分成每批100个样本

**validation_inputs, validation_targets = next(iter(validation_data))**

validation_data 是一个 dataset，每次迭代返回 (images, labels)。因为前面加载 MNIST 时用了tfds.load。参数 as_supervised=True 表示数据会以 (input, label)的形式返回：

input → 一张图像 (28x28x1 的 tensor)

label → 这张图像对应的数字 (0~9 的整数)

再加上后面做了 .batch()，所以 每次迭代的时候不是返回一张图，而是返回一批数据：

images → 一批图片，形状大概是 (batch_size, 28, 28, 1)

labels → 一批对应的标签，形状 (batch_size,)

**next(iter())**

iter是生成一个迭代器，然后让next从迭代器里取出一批一批地按顺序读取数据

next() 取出一个批次 (images, labels)。

iter(validation_data) → 生成一个迭代器，表示“按批次读取验证集”。

# **Outline the model**

In [18]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


**input_size：** 每张图片 28x28 = 784 个像素。

**output_size：** 模型输出 10 个节点，对应数字 0~9 的类别概率。

**hidden_layer_size：** 隐藏层有 50 个神经元，可以调节网络容量。

**tf.keras.layers.Flatten(input_shape=(28,28,1))**

将 28x28x1 的图像拉成 784 维向量。

MLP 的输入必须是一维向量，所以需要 Flatten。每一层的神经元是全连接的，也就是说上一层的每个输出都要连接到下一层的每个神经元。

如果输入是二维（28x28 图片），网络不知道怎么直接全连接，所以必须把它“拉平”成 一维向量（784 维）。

Flatten 层就是把 28x28x1 → 784，就可以送进全连接层。

**activation='relu'**

ReLU（Rectified Linear Unit）非线性激活函数

特点：

正数保持不变，负数变 0

训练快，解决梯度消失问题

适用场景：

隐藏层神经元，大多数深度神经网络默认用 ReLU

CNN、MLP 等隐藏层

**activation='softmax'**

**概率分布**

MNIST 是多分类问题（数字 0–9 共 10 类）

Softmax 会把每个神经元的输出转成 0~1 的概率，并且 所有输出加起来等于 1

这样可以直接用交叉熵损失函数（sparse_categorical_crossentropy）训练

**方便判断类别**

输出概率最大的那个神经元就是预测的类别。

例如 [0.01, 0.02, 0.90, ...]（概率 0.9 最大）。

# **Choose the optimizer and the loss function**

In [20]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

**optimizer='adam'** 自适应梯度下

Adam自适应学习率：每个参数都有自己的学习率，自动调整。动量机制：结合过去梯度的信息，加速收敛。

**loss='sparse_categorical_crossentropy'**  计算衡量误差

通过One-hot预测出概率后，用Softmax 会把每个神经元的输出转成 0~1 的概率，并且 所有输出加起来等于 1。最后用sparse_categorical_crossentropy计算衡量 Softmax 输出概率和真实类别之间差距，用于训练模型

直观理解

交叉熵 = 衡量“预测概率分布和真实标签分布的差距”

预测越接近真实类别 → 损失越小

预测离真实类别越远 → 损失越大

# **Training**

In [21]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 10s - 19ms/step - accuracy: 0.8836 - loss: 0.4082 - val_accuracy: 0.9385 - val_loss: 0.2130
Epoch 2/5
540/540 - 10s - 18ms/step - accuracy: 0.9471 - loss: 0.1819 - val_accuracy: 0.9520 - val_loss: 0.1607
Epoch 3/5
540/540 - 6s - 11ms/step - accuracy: 0.9592 - loss: 0.1378 - val_accuracy: 0.9600 - val_loss: 0.1347
Epoch 4/5
540/540 - 6s - 10ms/step - accuracy: 0.9666 - loss: 0.1132 - val_accuracy: 0.9690 - val_loss: 0.1069
Epoch 5/5
540/540 - 4s - 8ms/step - accuracy: 0.9712 - loss: 0.0948 - val_accuracy: 0.9743 - val_loss: 0.0946


**Epoch** = 训练整个数据集一次

**NUM_EPOCHS = 5** → 整个训练集会被完整训练 5 次

**train_data** → 训练集（已经 batch 好，每批 100 张图像）

**epochs=NUM_EPOCHS** → 训练轮数(5)

**validation_data=(validation_inputs, validation_targets)** → 验证集，用于每轮训练后评估模型效果，不参与训练

**verbose=2** → 输出训练进度：

每个 epoch 输出训练和验证的 loss/accuracy

# **Test the model**

In [22]:
test_loss, test_accuracy = model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9667 - loss: 0.1056


In [23]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy * 100))

Test loss: 0.11. Test accuracy: 96.67%


**test_data** → 测试集（未参与训练和验证）

**evaluate()** 会：

1. 前向传播整个测试集

2. 计算损失（test_loss）

3. 计算准确率（test_accuracy）

测试集指标能反映模型的 泛化能力（在新数据上的表现）